In [25]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import psycopg2
from psycopg2 import sql
import sqlite3
from datetime import datetime

In [15]:
topics = [
    {
        'category' : 'Technology',
        'title' : 'Data_engineering'
    },
     {
         'category' : 'Finance',
         'title' : 'Cryptocurrency'
     }
]

In [16]:
url = 'https://en.wikipedia.org/wiki/'

updated_data = []

In [17]:
for topic in topics:
    topic_url = url + topic.get('title')
    response = requests.get(topic_url)
    
    if response.status_code == 200:
        topic_soup = BeautifulSoup(response.content, 'html.parser')
        article_title = topic_soup.find('span', class_='mw-page-title-main').text
        last_modified = topic_soup.find('li', id='footer-info-lastmod').text
        author = "Wikipedia"
        content = topic_soup.find('div', id='bodyContent').text.strip()
        
        data_map = {
            "article_title": article_title,
            "last_updated": last_modified,
            "author": author,
            "category": topic.get('category'),
            "content": content,
            "source": topic_url
        }
        
        updated_data.append(data_map)

In [18]:
print(len(updated_data))


updated_data[1].get("article_title")


2


'Cryptocurrency'

#### Inserting the Data into an SQLite DB

In [19]:
# Connect to SQLite database (or create it if it doesn't exist)
conn = sqlite3.connect('knowledge_track.db')
cursor = conn.cursor()

# Create table
cursor.execute('''
CREATE TABLE IF NOT EXISTS articles (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    article_title TEXT,
    last_updated TIMESTAMP,
    author TEXT,
    category TEXT,
    content TEXT,
    source TEXT
)
''')

# Commit the changes and close the connection
conn.commit()

In [20]:
# Insert data into the table
cursor.execute('''
    INSERT INTO articles (article_title, last_updated, author, category, content, source)
    VALUES (:article_title, :last_updated, :author, :category, :content, :source)
''', data_map)

# Commit the changes and close the connection
conn.commit()

In [22]:
# Query to select all records from the articles table
cursor.execute('SELECT * FROM articles')

# Fetch all results
rows = cursor.fetchall()

# Print the results
for row in rows:
    for field in row:
        print(field)
    print("-" * 40)  # Separator between records


1
Cryptocurrency
 This page was last edited on 13 March 2025, at 20:20 (UTC).
Wikipedia
Finance
From Wikipedia, the free encyclopedia


Digital currency not reliant on a central authority
Not to be confused with Virtual currency.


A logo for Bitcoin, the first decentralized cryptocurrency
The genesis block of Bitcoin's blockchain, with a note containing The Times newspaper headline. This note has been interpreted as a comment on the instability caused by fractional-reserve banking.[1]: 18 
A cryptocurrency (colloquially crypto) is a digital currency designed to work through a computer network that is not reliant on any central authority, such as a government or bank, to uphold or maintain it.[2]
Individual coin ownership records are stored in a digital ledger or blockchain, which is a computerized database that uses a consensus mechanism to secure transaction records, control the creation of additional coins, and verify the transfer of coin ownership.[3][4][5] The two most common cons

In [23]:
# Close the connection
conn.close()

### Inserting the Data into a Postgres Database

In [29]:
# Connect to PostgreSQL database
conn = psycopg2.connect(
    dbname="knowledge_track",
    user="postgres",
    password="password",
    host="localhost",
    port="5432"
)

cursor = conn.cursor()

# Commit the changes
conn.commit()

In [30]:
# Create table
cursor.execute('''
CREATE TABLE IF NOT EXISTS articles (
    id SERIAL PRIMARY KEY,
    article_title TEXT,
    last_updated TIMESTAMP,
    author TEXT,
    category TEXT,
    content TEXT,
    source TEXT
)
''')

# Commit the changes
conn.commit()

In [ ]:
# Insert data into the table
cursor.execute('''
INSERT INTO articles (article_title, last_updated, author, category, content, source)
VALUES (%(article_title)s, %(last_updated)s, %(author)s, %(category)s, %(content)s, %(source)s)
''', data_map)

# Commit the changes
conn.commit()

InFailedSqlTransaction: current transaction is aborted, commands ignored until end of transaction block


In [ ]:
# Close the connection
cursor.close()
conn.close()